activate conda env keras.yml

In [ ]:
import keras, numpy as np, os, pandas as pd, time, random
import subprocess, re, sys, os
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras import layers
from keras.layers.core import Lambda
from keras import backend as K
from keras.models import Model
import tensorflow
from keras import metrics

In [42]:
#Set defaults 
seed = 374173
latent_dim = 3
weight = 1000
k = 8
train_prop = 0.9
depth = 6
width = 128
patience = 100
batch_size=32
prediction_freq=2
max_epochs = 300

In [44]:
out = 'training/'
KMER = '../../data/kmers/GCrefv1_kmers.npy'
META = '../../data/metadata/GCrefv1.tsv'
NAMES = '../../data/GCrefv1_samples.csv'

In [45]:
data = np.load(KMER)
names = pd.read_csv(NAMES)['sampleID']
meta = pd.read_csv(META, sep='\t')

In [46]:
print(data.shape, names.shape, meta.shape)

(782, 65536) (782,) (782, 2)


In [47]:
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tensorflow.set_random_seed(seed)

In [48]:
#split training and test data
ninds = data.shape[0]
train = np.random.choice(range(ninds), int(train_prop*ninds), replace=False)
test = np.array([x for x in range(ninds) if x not in train])
traindata=data[train,:]
testdata=data[test,:]
trainsamples=names.loc[train]
testsamples=names.loc[test]

### Set up model

In [49]:
#load model
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                             mean=0., stddev=1.,seed=seed)
    return z_mean + K.exp(z_log_var) * epsilon

In [50]:
#encoder
input_seq = keras.Input(shape=(traindata.shape[1],))
x=layers.Dense(width,activation="elu")(input_seq)
for i in range(depth-1):
    x=layers.Dense(width,activation="elu")(x)
z_mean=layers.Dense(latent_dim)(x)
z_log_var=layers.Dense(latent_dim)(x)
z = layers.Lambda(sampling,output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
encoder=Model(input_seq,[z_mean,z_log_var,z],name='encoder')

In [51]:
#decoder
decoder_input=layers.Input(shape=(latent_dim,),name='z_sampling')
x=layers.Dense(width,activation="linear")(decoder_input)
for i in range(depth-1):
    x=layers.Dense(width,activation="elu")(x)
output=layers.Dense(traindata.shape[1],activation="softplus")(x) #can also use other activations that return strictly positive lambda parameter values
decoder=Model(decoder_input,output,name='decoder')

In [52]:
#combine for complete vae
output_seq = decoder(encoder(input_seq)[2])
vae = Model(input_seq, output_seq, name='vae')

In [53]:
#define loss function
#likelihood term is Poisson loss
#clipped to avoid floating point errors in logarithm
reconstruction_loss = output_seq - input_seq - input_seq * K.log(output_seq) + input_seq * K.log(K.clip(input_seq, 1e-07, data.max()))
reconstruction_loss = K.sum(reconstruction_loss, axis=-1)
#regularisation term is (twice?) the negative of the KL divergence of a diagonal multivariate normal and a standard normal
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var) 
kl_loss = K.sum(kl_loss, axis=-1)
#regularisation term is downweighed by weight parameter
kl_loss *= -1/weight/weight/traindata.shape[1] 
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

In [54]:
vae.compile(optimizer='adam')

/Users/mariloumercedes/miniconda3/envs/keras/lib/python3.6/site-packages/keras/engine/training_utils.py:819: UserWarning: Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
  'be expecting any data to be passed to {0}.'.format(name))


In [55]:
#callbacks
#save only weights for the best performing model
#according to validation loss
checkpointer=keras.callbacks.ModelCheckpoint(
              filepath=out+"_weights.hdf5",
              verbose=0,
              save_best_only=True,
              save_weights_only=True,
              monitor="val_loss",
              period=1)

#stop if no improvement for 'patience' number of updates
earlystop=keras.callbacks.EarlyStopping(monitor="val_loss",
                                        min_delta=0,
                                        patience=patience)

#half learning rate if no improvement for .25*patience number of updates
reducelr=keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                           factor=0.5,
                                           patience=int(patience/4),
                                           verbose=1,
                                           mode='auto',
                                           min_delta=0,
                                           cooldown=0,
                                           min_lr=0)

In [57]:
print_predictions=keras.callbacks.LambdaCallback(
         on_epoch_end=lambda epoch,
         logs:saveLDpos(encoder=encoder,
                        predgen=data,
                        samples=names,
                        batch_size=batch_size,
                        epoch=epoch,
                        frequency=prediction_freq))

In [58]:
#training
t1=time.time()
history=vae.fit(x=traindata, 
               y=None,
               shuffle=True,
               epochs=300, 
               callbacks=[checkpointer,earlystop,reducelr,print_predictions],
               validation_data=(testdata,None), 
               batch_size=32)
t2=time.time()
vaetime=t2-t1
print("VAE run time: "+str(vaetime)+" seconds")

Train on 703 samples, validate on 79 samples
Epoch 1/300
703/703 [==============================] - 4s 6ms/step - loss: 15956.0348 - val_loss: 3363.4790
Epoch 2/300
703/703 [==============================] - 4s 5ms/step - loss: 2972.7873 - val_loss: 3014.0432
Epoch 3/300
703/703 [==============================] - 4s 5ms/step - loss: 2776.8832 - val_loss: 2986.1022
Epoch 4/300
703/703 [==============================] - 4s 5ms/step - loss: 2754.0377 - val_loss: 2981.7470
Epoch 5/300
703/703 [==============================] - 4s 5ms/step - loss: 2753.7825 - val_loss: 3004.4153
Epoch 6/300
703/703 [==============================] - 4s 5ms/step - loss: 2754.7493 - val_loss: 2990.1944
Epoch 7/300
703/703 [==============================] - 4s 5ms/step - loss: 2746.1221 - val_loss: 2973.3034
Epoch 8/300
703/703 [==============================] - 4s 5ms/step - loss: 2742.0581 - val_loss: 2959.4346
Epoch 9/300
703/703 [==============================] - 4s 5ms/step - loss: 2742.8765 - val_loss: 2

In [59]:
#save training history
h=pd.DataFrame(history.history)
h.to_csv(out+"_history.txt")

## Project to latent space

In [60]:
#predict latent space coords for all samples from weights minimizing val loss
vae.load_weights(out+"_weights.hdf5")
pred=encoder.predict(data,batch_size=batch_size) 
p=pd.DataFrame()

p['mean1']=pred[0][:,0]
p['mean2']=pred[0][:,1]
p['mean3']=pred[0][:,2]
p['sd1']=pred[1][:,0]
p['sd2']=pred[1][:,1]
p['sd3']=pred[1][:,2]
pred=p

pred['sampleID']=names

In [61]:
pred.loc[pred.sampleID.isin(testsamples), "status"] = "validation"
pred.loc[pred.status.isnull(), "status"] = "training"
meta_dict = dict(zip(meta.s_Sample, meta.species))
pred['species'] = pred.sampleID.map(meta_dict)
pred.to_csv(out+'latent_coords.csv',index=False)